# Config

In [58]:
import os
import google.generativeai as genai
import time
import json
import random

In [59]:
genai.configure(api_key="")

In [60]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

In [61]:
# Define the model
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [78]:
countries = [
    "Singapore",
    "United States",
    "United Kingdom",
    "Australia",
    "Canada",
    "New Zealand",
]

# For Scam

In [86]:
scam_list = [
    "Bank Impersonation Scam – Scammers pretend to be from your bank, claiming there's suspicious activity on your account.",
    "Credit Card Fraud Scam – Callers claim your credit card was used fraudulently and ask for verification details.",
    "Loan Approval Scam – Fake lenders offer 'pre-approved' loans but require an upfront payment or personal information.",
    "Debt Collection Scam – Fake debt collectors demand immediate payment for debts you don’t owe.",
    "Student Loan Forgiveness Scam – Callers promise to erase student loans in exchange for fees or personal data.",
    "IRS/Tax Scam – Scammers impersonate tax agencies and demand immediate payment under threat of arrest.",
    "Social Security Number (SSN) Scam – Callers claim your SSN is compromised and need verification.",
    "Medicare/Health Insurance Scam – Scammers pretend to be from Medicare, asking for policy details or payments.",
    "Jury Duty Scam – Fake legal representatives claim you missed jury duty and must pay a fine.",
    "Customs & Border Protection Scam – Callers claim you have illegal packages in your name and must pay to avoid charges.",
    "Microsoft/Apple Support Scam – Scammers claim your computer is infected and require remote access.",
    "Amazon Order Scam – Callers say there’s a suspicious order on your Amazon account and request verification.",
    "Google Business Listing Scam – Scammers say your business listing is about to be removed unless you pay.",
    "Antivirus Subscription Scam – Callers pretend to be from antivirus companies, offering fake renewals.",
    "Internet Provider Scam – Scammers claim your internet is being shut down unless you verify payment.",
    "Fake Sweepstakes/Lottery Scam – Scammers say you won a prize but must pay a 'processing fee' first.",
    "Publisher’s Clearing House Scam – Impersonators claim you're a winner and need to pay taxes upfront.",
    "Vacation Prize Scam – Scammers offer free trips but require personal details or payment.",
    "Fake Charity Scam – Callers ask for donations to fake charities, often during crises.",
    "Secret Shopper Scam – Callers recruit victims as 'secret shoppers' but require fees to participate.",
    "Work-From-Home Scam – Scammers offer fake remote jobs requiring upfront payments.",
    "Fake Job Interview Scam – Callers claim to offer high-paying jobs and ask for personal data.",
    "Pyramid Scheme Scam – Callers promote 'investment opportunities' that require recruitment of others.",
    "Business Grant Scam – Scammers promise government grants for businesses but demand fees.",
    "Invoice Fraud Scam – Fake suppliers claim unpaid invoices and demand payment.",
    "Online Dating Scam – Romance scammers build relationships online and ask for money.",
    "Military Romance Scam – Scammers impersonate soldiers asking for financial help.",
    "Inheritance Scam – Callers claim you inherited money but must pay a fee to receive it.",
    "Fake Relative in Distress Scam – Scammers pretend to be a family member needing urgent help.",
    "Wedding Vendor Scam – Fake vendors take deposits for weddings and disappear.",
    "Electricity/Water Shutoff Scam – Scammers say your utilities will be shut off unless you pay.",
    "Solar Panel Scam – Fake companies offer government rebates on solar panels and ask for personal details.",
    "Home Repair Scam – Callers claim your home needs urgent repairs and demand deposits.",
    "Duct Cleaning Scam – Callers offer cheap duct cleaning services but demand upfront payment.",
    "Property Tax Scam – Fake assessors claim your property tax needs immediate payment.",
    "Health Insurance Refund Scam – Callers claim you're due a refund but need account details.",
    "Pain Relief Device Scam – Fake medical suppliers offer free pain relief devices in exchange for Medicare info.",
    "COVID-19 Vaccine/Testing Scam – Scammers offer fake COVID tests or vaccines for money.",
    "Prescription Drug Scam – Fake pharmacies offer discounted medicine and steal payment info.",
    "Disability Benefits Scam – Callers claim you qualify for disability benefits but need verification fees.",
    "Fake Stock Investment Scam – Scammers promise high investment returns and steal money.",
    "Cryptocurrency Scam – Callers convince victims to invest in fake crypto schemes.",
    "Ponzi Scheme Scam – Fraudsters recruit investors for nonexistent businesses.",
    "Fake Gold/Precious Metals Investment Scam – Scammers claim to sell gold at 'discounted' rates.",
    "Forex Trading Scam – Callers promote fake forex trading platforms and steal investments.",
    "Police/Legal Threat Scam – Scammers pretend to be police and demand fines.",
    "Fake Survey Scam – Callers ask personal questions to steal information.",
    "Pet Adoption Scam – Scammers offer non-existent pets for adoption fees.",
    "Rental Property Scam – Fake landlords demand security deposits for properties they don’t own.",
    "Grandparent Scam – Scammers pretend to be grandchildren needing emergency money."
]

In [87]:
# File where JSON logs will be stored
output_file = "scam_call_logs.json"

# Read existing data if the file exists
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        try:
            all_logs = json.load(f)
            if not isinstance(all_logs, list):  # Ensure it's a list
                all_logs = []
        except json.JSONDecodeError:
            all_logs = []
else:
    all_logs = []  # Initialize empty list if file does not exist

# For each scam category, generate logs
for item in scam_list:
    print(f"Generating logs for: {item}")
    for i in range(1): # ⚠️⚠️ NEED TO CHANGE: SET TO 1 JUST FOR TESTING PURPOSE
        # Generate
        response = model.generate_content(""" 
                    Create a full and complete theoretical scam call log in JSON format for the purpose of learning.
                    You are free to determine the usual length of each scam call log, the longer the better.
                    The log should contain ONLY the spoken dialogue between the caller and receiver.
                    Do not include any descriptions of actions, such as hanging up, typing, or any other non-verbal cues.
                    All dialogue should be a verbatim transcription of a hypothetical conversation.
                    All examples should be valid scams, is_scam should be true.
                    A victim is considered "scammed"=true if they have given out any PII.
                    Follow the JSON format below.
                    The context of this scam takes place in""" + random.choice(countries) + """. But only use English words and language.

                    The call may or may not contain PII (Personal Info) from the list below.
                    If it does, use placeholders wrapped in square brackets. Do not include any PII not in the examples.
                    [BANK PIN] [BANK ACCOUNT NUMBER] [PHONE NUMBER]
                    [CARD NUMBER] [CVV] [CARD EXPIRY DATE]
                    [DATE OF BIRTH] [SSN] [LOGIN ID] [PASSWORD]

                    If you include any names, just use a fictitious name of someone living in that country.

                    For context, scammers usually elicit this information by:
                    Impersonation: Posing as legitimate organizations (government agencies, banks, tech companies, etc.) to gain your trust.
                    Threats: Claiming you owe money, face legal consequences, or have a security issue.
                    Promises: Offering prizes, discounts, or services in exchange for your information.
                    Empathy/Urgency: Creating a sense of crisis to pressure you into acting quickly without thinking.
                    Building Rapport: Trying to be friendly and conversational to lower your guard.
                    Technical Jargon: Using confusing language to make you feel like you need their help.
                    Spoofing: Making their caller ID appear legitimate.

                    The category of the scam call log should be:
                    """ + random.choice(scam_list) + """

                    Caller is the Scammer
                    Receiver is the Victim

                    Please follow this JSON format strictly.
                    Do not add additional information or modify the structure.

                    [
                    {
                        "scam_category": "e.g. bank",
                        "is_scam": true/false,
                        "scammed": true/false,
                        "language": "",
                        "dialogue": [
                        {
                            "person": "caller",
                            "message": "Hello I am ..."
                        },
                        {
                            "person": "receiver",
                            "message": "Yes this is..."
                        },
                        {
                            "person": "caller",
                            "message": "We need your bank pin urgently..."
                        },
                        {
                            "person": "receiver",
                            "message": "Sure my bank pin is [BANK PIN]"
                        }
                        ... the rest of the conversation (The victim may or may not have given the information (scammed or not scammed))
                        ]
                    }
                    ]
                    """
                    )

        try:
            # Workaround for model returning malformed JSON
            response_text = response.text.replace("}]}]}", "}]}]")
            
            # Parse response as JSON
            new_logs = json.loads(response_text)

            # Ensure the new logs are in list format
            if isinstance(new_logs, list):
                all_logs.extend(new_logs)  # Append new data to existing list
            else:
                print(f"Iteration {i+1}: Unexpected JSON format, skipping.")

            # Save updated logs back to file
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(all_logs, f, ensure_ascii=False, indent=4)

            print(f"Iteration {i+1}: Log successfully added.")

        except json.JSONDecodeError as e:
            print(f"Iteration {i+1}: Failed to parse JSON - {e}")

        # Wait 10 seconds before the next iteration
        time.sleep(10)

print("Process completed.")

Generating logs for: Bank Impersonation Scam – Scammers pretend to be from your bank, claiming there's suspicious activity on your account.
Iteration 1: Log successfully added.
Generating logs for: Credit Card Fraud Scam – Callers claim your credit card was used fraudulently and ask for verification details.
Iteration 1: Log successfully added.
Generating logs for: Loan Approval Scam – Fake lenders offer 'pre-approved' loans but require an upfront payment or personal information.
Iteration 1: Log successfully added.
Generating logs for: Debt Collection Scam – Fake debt collectors demand immediate payment for debts you don’t owe.
Iteration 1: Log successfully added.
Generating logs for: Student Loan Forgiveness Scam – Callers promise to erase student loans in exchange for fees or personal data.
Iteration 1: Log successfully added.
Generating logs for: IRS/Tax Scam – Scammers impersonate tax agencies and demand immediate payment under threat of arrest.
Iteration 1: Log successfully added

KeyboardInterrupt: 

# For Non-Scam

In [85]:
legitimate_list = [
    "Family Check-In – A relative calls to see how you're doing.",
    "Friend Catch-Up – A friend calls just to chat.",
    "Birthday Call – Someone calls to wish you a happy birthday.",
    "Holiday Greeting Call – A call from friends or family during holidays.",
    "Event Invitation – A friend invites you to a party, wedding, or gathering.",
    "Apology Call – Someone calls to make amends after a disagreement.",
    "Surprise Call – An unexpected but welcome call from an old friend.",
    "Condolence Call – Someone calls to express sympathy after a loss.",
    "New Baby Announcement – A family member calls to share exciting baby news.",
    "Engagement/Wedding Announcement – A friend or relative shares engagement or wedding news.",
    "Job Interview Call – A recruiter or hiring manager schedules an interview.",
    "Client Inquiry Call – A potential client calls for more information about your services.",
    "Business Meeting Call – A colleague calls to coordinate a meeting.",
    "Sales Call (Legitimate) – A representative calls about a service or product you’re genuinely interested in.",
    "Project Update Call – A team member provides updates on a joint project.",
    "Boss/Supervisor Check-In – A manager calls to discuss work tasks.",
    "Coworker Collaboration Call – A coworker calls to discuss a work-related matter.",
    "Vendor or Supplier Call – A supplier calls about an order or service.",
    "Work-from-Home Coordination Call – A call to align remote work expectations.",
    "HR or Payroll Call – A legitimate HR rep calls about benefits, salary, or work policies.",
    "Bank Fraud Alert – Your bank calls to verify suspicious activity on your account.",
    "Doctor’s Office Call – A clinic or hospital calls about an appointment or test results.",
    "Dentist Appointment Reminder – A dentist's office confirms your scheduled appointment.",
    "Pharmacy Prescription Ready Call – A pharmacy notifies you that your medication is ready.",
    "Insurance Verification Call – Your health or auto insurance provider confirms details.",
    "Home Repair Technician Call – A plumber, electrician, or contractor calls about scheduled repairs.",
    "Cable/Internet Service Call – Your provider calls to confirm a service upgrade or maintenance.",
    "Mechanic or Car Service Call – An auto repair shop updates you on your vehicle’s status.",
    "Package Delivery Call – A courier service calls to confirm or update you on a delivery.",
    "Tech Support Follow-Up – A legitimate company calls back after a support request.",
    "School or University Call – A teacher or administrator calls regarding your child’s education.",
    "Library Book Due Reminder – A public library calls to remind you of overdue books.",
    "Community Event Call – A local organization invites you to an event.",
    "PTA Meeting Call – A parent-teacher association rep calls about school events.",
    "Alumni Association Call – Your former school or university reaches out for networking or fundraising.",
    "Tutoring Session Confirmation – A tutor or educational service calls to confirm an appointment.",
    "Volunteer Coordination Call – A non-profit or charity calls about volunteer work.",
    "Church or Religious Group Call – A faith-based group contacts you for updates or support.",
    "Sports Team Call – A coach or team member calls to confirm game details.",
    "Club or Hobby Group Call – A book club, running group, or other interest-based club calls to coordinate events.",
    "Police Department Notification – Law enforcement calls for legitimate reasons (e.g., found property, crime update).",
    "Fire Department Call – A fire department provides updates or safety alerts.",
    "Ambulance or Medical Emergency Call – A hospital or paramedic provides critical medical updates.",
    "City or County Alert Call – Local government contacts you about weather emergencies, road closures, or safety issues.",
    "Jury Duty Notification Call – A court calls to confirm your jury duty service.",
    "Social Services Call – A legitimate agency calls about benefits or assistance programs.",
    "Voter Registration Call – Election officials provide voting details or updates.",
    "Military Service Call – A recruiter or officer calls regarding military service or benefits.",
    "Public Utility Call – Your water, gas, or electric company notifies you of outages or maintenance.",
    "Legal Call from an Attorney – A lawyer calls about an ongoing case or legal matter."
]

In [ ]:
# File where JSON logs will be stored
output_file = "non_scam_call_logs.json"

# Read existing data if the file exists
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        try:
            all_logs = json.load(f)
            if not isinstance(all_logs, list):  # Ensure it's a list
                all_logs = []
        except json.JSONDecodeError:
            all_logs = []
else:
    all_logs = []  # Initialize empty list if file does not exist

for item in legitimate_list:
    print("Generating logs for", item)
    for i in range(1): # ⚠️⚠️ NEED TO CHANGE: SET TO 1 JUST FOR TESTING PURPOSE
        response = model.generate_content(""" 
                    Create full and complete theoretical call logs in JSON format.
                    The call is a valid conversation between two people in a real-world scenario.
                                        
                    You are free to determine the appropriate length of each call log, the longer the better.
                    The context of this call takes place in""" + random.choice(countries) + """.
                    No need to include any non-verbal cues.
                    Only use English words and language found in the dictionary.

                    The log should contain ONLY the spoken dialogue between the caller and receiver.
                    Do not include any descriptions of actions, such as hanging up, typing, or any other non-verbal cues.
                    All dialogue should be a verbatim transcription of a hypothetical conversation.

                    The category of the call log should be: """ + random.choice(legitimate_list) + """
                    
                    Ensure that the category is properly represented in the JSON.

                    The call may or may not contain PII (Personal Info) from the list below.
                    If it does, use placeholders wrapped in square brackets, e.g. [PHONE NUMBER]
                    If you include any names, just use a fictitious name of someone living in that country, do not use placeholders for names.
                                        
                    Caller is the one who intiaited the call
                    Receiver is the person answering the call

                    Please follow this JSON format strictly.
                    Do not add additional information or modify the structure.
                    Make sure to check the brackets and quotes in the JSON format.                      

                    [
                    {
                        "call_category": "",
                        "language": "",
                        "dialogue": [
                        {
                            "person": "caller",
                            "message": "Hello, this is..."
                        },
                        {
                            "person": "receiver",
                            "message": "Oh, hi! How can I help you?"
                        },
                        {
                            "person": "caller",
                            "message": "I'm calling about..."
                        },
                        {
                            "person": "receiver",
                            "message": "Sure, let me check on that..."
                        }
                        ... the rest of the conversation
                        ]
                    }
                    """
                    )

        try:
            # Workaround for model returning malformed JSON
            response_text = response.text.replace("}]}]}", "}]}]")

            # Parse response as JSON
            new_logs = json.loads(response_text)

            # Ensure the new logs are in list format
            if isinstance(new_logs, list):
                all_logs.extend(new_logs)  # Append new data to existing list
            else:
                print(response.text)
                print(f"Iteration {i+1}: Unexpected JSON format, skipping.")

            # Save updated logs back to file
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(all_logs, f, ensure_ascii=False, indent=4)

            print(f"Iteration {i+1}: Log successfully added.")

        except json.JSONDecodeError as e:
            print(response.text)
            print(f"Iteration {i+1}: Failed to parse JSON - {e}")

        # Wait 10 seconds before the next iteration
        time.sleep(10)

print("Process completed.")

Generating logs for Family Check-In – A relative calls to see how you're doing.
Iteration 1: Log successfully added.
Generating logs for Friend Catch-Up – A friend calls just to chat.
Iteration 1: Log successfully added.
Generating logs for Birthday Call – Someone calls to wish you a happy birthday.
Iteration 1: Log successfully added.
Generating logs for Holiday Greeting Call – A call from friends or family during holidays.
Iteration 1: Log successfully added.
Generating logs for Event Invitation – A friend invites you to a party, wedding, or gathering.
{"call_category": "Friend Catch-Up", "language": "English", "dialogue": [{"person": "caller", "message": "G'day, mate! It's Liam."}, {"person": "receiver", "message": "Liam! Hey, how's it going?  Long time no talk!"}, {"person": "caller", "message": "Yeah, it has been a while!  Life's been pretty hectic.  Just wanted to see how you were doing."}, {"person": "receiver", "message": "Not too bad, thanks.  Busy with work, you know how it i

KeyboardInterrupt: 

# Prepare for Processing

In [89]:
# Load the JSON for the scam call logs
with open("scam_call_logs.json", "r", encoding="utf-8") as f:
    scam_logs = json.load(f)

# Load the JSON for the non-scam call logs
with open("non_scam_call_logs.json", "r", encoding="utf-8") as f:
    non_scam_logs = json.load(f)

In [109]:
import re

scam_pii = []
for log in scam_logs:
    for dialogue in log["dialogue"]:
        if "message" in dialogue:
            message = dialogue["message"]
            pii_matches = re.findall(r"\[[^\]]+\]", message)  # Matches anything inside square brackets
            scam_pii.extend(pii_matches)

# Keep only unique PII entries
scam_pii = list(set(scam_pii))

In [110]:
for i in scam_pii:
    print(i)

[BANK PIN]
[Phone Number]
[PHONE NUMBER]
[FULL ADDRESS]
[LOGIN ID]
[AMOUNT]
[BANK NAME]
[Name]
[SSN]
[Full Address]
[Business Name]
[NATIONAL INSURANCE NUMBER]
[SORT CODE]
[DOMAIN]
[CARD NUMBER]
[CARD EXPIRY DATE]
[CVV]
[BANK ACCOUNT NUMBER]
[DATE OF BIRTH]


In [111]:
import re

non_scam_pii = []
for log in non_scam_logs:
    for dialogue in log["dialogue"]:
        if "message" in dialogue:
            message = dialogue["message"]
            pii_matches = re.findall(r"\[[^\]]+\]", message)  # Matches anything inside square brackets
            non_scam_pii.extend(pii_matches)

# Keep only unique PII entries
non_scam_pii = list(set(non_scam_pii))

In [112]:
for i in non_scam_pii:
    print(i)

[Address]
[Client Contact Name]
[Policy Number]
[Phone Number]
[PHONE NUMBER]
[Date of Birth]
[ADDRESS]
[Receiver's Name]
[Date]
[Previous Company Name]
[Time]
[Make]
[Specific Maple Syrup Brand]
[Deductible]
[Department Name]
[Website Address]
[Model]
[Year]
[Email Address]
[email address]
